# Webhose Watson Discovery Ingestion

In [ ]:
%%capture
!pip install ibm-watson
!pip install webhoseio

In [ ]:
import webhoseio
import json
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
authenticator = IAMAuthenticator('{discovery-api-key}')
discovery = DiscoveryV1(
    version='2019-04-30',
    authenticator=authenticator
)

discovery.set_service_url('{discovery-url}')

In [ ]:
# Query for competitor queries

# add your webhose token below
webhoseio.config(token="put-your-webhose-token-here")
query_params = {
  "q": "\"phrase one\" OR \"phrase two\" language:english",
  "sort": "crawled"
}

output = webhoseio.query("filterWebContent", query_params)

total = int(output['totalResults'])
print("Total results: ", total)

In [ ]:
for post in output['posts']:
    docId = post['uuid']

    # print(json.dumps(post, indent=2))

    if post['thread']['site_type'] == "news" and post['author'] != "" and 'adult' not in post['thread']['site_categories']:
        # remove pre-enriched data
        if 'entities' in post.values():
            del post['entities']

        # print(json.dumps(post, indent=2))

        add_doc = discovery.update_document(
            '{discovery-environment-id}',
            '{discovery-collection-id}',
            docId,
            file_content_type='application/json',
            filename=docId + '.json',
            file=json.dumps(post)).get_result()
        print(json.dumps(add_doc, indent=2))

output = webhoseio.get_next()